In [9]:
import numpy as np
import pandas as pd
import pickle


In [ ]:
import json
import re
import sys
import time
import warnings
from random import sample

import openai
from openai import AzureOpenAI
import pandas as pd
# from tqdm import tqdm
import tiktoken
key = "insert_key_here" 

# Parameters
client = AzureOpenAI(
  azure_endpoint = "https://hkust.azure-api.net",
  api_version = "2024-06-01",
  api_key = key #put your api key here
)

# Function
def get_response(message, instruction):
    response = client.chat.completions.create(
		model = 'gpt-4',
        temperature = 1,
        messages = [
            {"role": "system", "content": instruction},
            {"role": "user", "content": message}
        ]
    )
    
    # print token usage
    print(response.usage)
    # return the response
    return response.choices[0].message.content
  
def LM_generation(prompt, max_tokens = 10, temperature = 0.1, model = "gpt-4", system_msg = None, msg_history = None):
    retry_attempt = 5
    retry_num = 0
    generation_success = False
    messages = []
    if system_msg is not None:
        messages.append({'role':'system', 'content': system_msg})
    if msg_history is not None:
        messages = msg_history
    messages.append({'role':'user', 'content':prompt})
    while retry_num < retry_attempt and not generation_success:
        try:
            gen = client.chat.completions.create(
                model = model,
                messages = messages,
                max_tokens = max_tokens,
                temperature = temperature,
            ) 
            generation_success = True
            input_tokens = gen.usage.prompt_tokens
            output_tokens = gen.usage.completion_tokens
        except openai.APIError as e:
            retry_num += 1
            generation_success = False
            time.sleep(5)
            # return "openai.error.APIError", -1, -1 
        except openai.RateLimitError as e:
            retry_num += 1
            generation_success = False
            time.sleep(30)
        except:
            retry_num += 1
            generation_success = False
            time.sleep(10)
    
    if generation_success:
        if gen == None or gen.choices[0] == None or gen.choices[0].message == None or gen.choices[0].message.content == None:
            return "", 0, 0
        return gen.choices[0].message.content.strip(), input_tokens, output_tokens
    else:
        return "Error", -1,-1

def count_tokens(text, encoding_name = "cl100k_base"):
    encoding = tiktoken.get_encoding(encoding_name)
    # print(text)
    num_tokens = len(encoding.encode(text))
    return num_tokens


In [ ]:
import json

# Open and read the JSON file
with open('llm_ready_opentom_vfin.jsonl') as f: # ambiguous story
    final_opentom = [json.loads(line) for line in f]
    
with open('llm_ready_bigtom_vfin.jsonl') as f: # ambiguous story
    final_bigtom = [json.loads(line) for line in f]

bigtom_filter = np.load('bigtom_filter.npy', allow_pickle=True)
opentom_filter = np.load('opentom_filter.npy', allow_pickle=True)

final_opentom = np.array(final_opentom)[opentom_filter]
final_bigtom = np.array(final_bigtom)[bigtom_filter]

In [5]:
#save

def save_list_of_dict(save_name, list_of_dict):
    with open(save_name, 'wb') as f:
        pickle.dump(list_of_dict, f)

def load_list_of_dict(save_name):
    with open(save_name, 'rb') as f:
        loaded = pickle.load(f)
    return loaded

In [ ]:
senses = ['visual', 'auditory', 'olfactory', 'gustatory', 'tactile', 'proprio']
num_senses = len(senses)
# thermos can be merged with tactile, but minor issue.

In [7]:
sense_dict = {senses[i]: i for i in range(num_senses)}

LLM Task 1:

In [ ]:
# Answer Holder - I think we should use list? Better to store answer, then process later.
# Can hold answers in a list of dict
# {ID, groundtruth_type, sense, option_order (B/A), llm_answer, true_answer}

In [8]:
bigtom_task1_ans = [{'ID': instance['ID'], 'groundtruth_classification': instance['groundtruth_classification']
                     , 'sense': instance['sense'],
                     'answer_order': [[0,1], [1,0],[0,1],[1,0]], # this is the f() that can map original order to postprocess order
                     'llm_answer': ['X','X','X','X'],
                     'true_answer': [instance['belief_change_label'], 1 - instance['belief_change_label'], instance['belief_change_label'], 1 - instance['belief_change_label']]} 
                    for instance in final_bigtom]
opentom_task1_ans = [{'ID': instance['ID'], 'groundtruth_classification': instance['groundtruth_classification']
                     , 'sense': instance['sense'],
                     'answer_order': [[0,1], [1,0],[0,1],[1,0]], # this is the f() that can map original order to postprocess order
                     'llm_answer': ['X','X','X','X'],
                     'true_answer': [instance['belief_change_label'], 1 - instance['belief_change_label'], instance['belief_change_label'], 1 - instance['belief_change_label']]} 
                    for instance in final_opentom]

In [9]:
# Prompts
bigtom_system = "Story: {story} \n\nEvent: {event} \n\n"
bigtom_system += "Regarding the event, the main character {protag_name} only encounters the following percept: {percept}\n\n"
bigtom_system += "Before the event, the main character's initial belief was \"{initial_belief}\"\n\n"
bigtom_system += "Instructions: Analyze the situation and select the most appropriate answer to the following question: \n\n\n"

opentom_system = "Story: {story} \n\n"
opentom_system += "After the story, the main character {protag_name} only encounters the following percept: {percept}\n\n"
opentom_system += "Before encountering the percept, the main character's initial belief was \"{initial_belief}\"\n\n"
opentom_system += "Instructions: Analyze the situation and select the most appropriate answer to the following question: \n\n\n"


# Needed variables: story, event, protag_name, percept, initial_belief

In [ ]:
bigtom_q1_prompt_1 = "After the event happened, based on the main character {protag_name} perception of the event, "
bigtom_q1_prompt_1 += "is the following statement about {protag_name}'s belief True or False: \n"
bigtom_q1_prompt_1 += "Statement: \"{initial_belief}\"\n"

bigtom_q1 = [bigtom_q1_prompt_1, bigtom_q1_prompt_1]
bigtom_q1[0] = bigtom_q1[0] + "A. True\nB. False"
bigtom_q1[1] = bigtom_q1[1] + "A. False\nB. True"

bigtom_q1_prompt_2 = "Did the event cause the main character {protag_name} to change their belief?\n"
bigtom_q1.append(bigtom_q1_prompt_2 + "A. No\nB. Yes")
bigtom_q1.append(bigtom_q1_prompt_2 + "A. Yes\nB. No")

opentom_q1_prompt_1 = "After encountering the percept, based on the main character {protag_name}'s perception, is the following statement about {protag_name}'s belief True or False: \n"
opentom_q1_prompt_1 += "Statement: \"{initial_belief}\"\n"
opentom_q1 = [opentom_q1_prompt_1, opentom_q1_prompt_1]
opentom_q1[0] = opentom_q1[0] + "A. True\nB. False"
opentom_q1[1] = opentom_q1[1] + "A. False\nB. True"

opentom_q1_prompt_2 = "Did the percept cause the main character {protag_name} to change their belief?\n"
opentom_q1.append(opentom_q1_prompt_2 + "A. No\nB. Yes")
opentom_q1.append(opentom_q1_prompt_2 + "A. Yes\nB. No")

# Needed variable: protag_name, initial_belief

In [11]:
option_dict = {'A': 0, 'B': 1, 'C': 2, 'D': 3}

# def post_randomize_order(val, n): 
#     # val = 
#     # n = number of randomize

bigtom_task2_ans = [{'ID': instance['ID'], 'groundtruth_classification': instance['groundtruth_classification']
                     , 'sense': instance['sense'],
                     'answer_order': [], # this is the f() that can map original order to postprocess order
                     'llm_answer': ['X','X','X','X','X'],
                     'true_answer': [],
                     'true_answer_og': option_dict[instance['new_belief_label']]} 
                    for instance in final_bigtom]
opentom_task2_ans = [{'ID': instance['ID'], 'groundtruth_classification': instance['groundtruth_classification']
                     , 'sense': instance['sense'],
                     'answer_order': [], # this is the f() that can map original order to postprocess order
                     'llm_answer': ['X','X','X','X','X'],
                     'true_answer': [],
                     'true_answer_og': option_dict[instance['new_belief_label']]}  
                    for instance in final_opentom]

In [12]:
import random
random.seed(0)
np.random.seed(0)
task2_ans = [bigtom_task2_ans, opentom_task2_ans]
for t in task2_ans:
    for instance in t:
        for _ in range(5):
            randomizer = np.random.choice(np.arange(4), 4, replace=False) # 0 --> a0, 1 --> a1
            instance['answer_order'].append(randomizer.tolist())
            # instance['true_answer'].append(['A','B','C','D'][randomizer[instance['true_answer_og']]])
            true_answer_idx = np.where(randomizer == instance['true_answer_og'])[0][0]
            instance['true_answer'].append(['A','B','C','D'][true_answer_idx])

In [14]:
# import numpy as np

# array = np.array([1, 2, 3, 4, 5])
# value = 3
# index = np.where(array == value)
# print(f"Index of value {value}: {index[0]}")

In [16]:
# bigtom_task2_ans

In [13]:
bigtom_q2_prompt = "{default_question}\n"
bigtom_q2_prompt += "Choose the most appropriate answer (choose one only):\n"
bigtom_q2_prompt += "A. {first}\nB. {second}\nC. {third}\nD. {fourth}"
opentom_q2_prompt = bigtom_q2_prompt

# "After the event happened, based on the main character Noor's perception of the event, which of the following statements best describes Noor's new belief?"

In [14]:
import json

def save_jsonl(file_name, path):
    with open(path, "w") as file:
        for item in file_name:
            json_string = json.dumps(item)
            file.write(json_string + '\n')


In [15]:
# Choose model
model = "gpt-35-turbo"

In [ ]:
# BigToM first - Task 1
model = "gpt-35-turbo"
# positives
count = 0
input_tokens = 0
output_tokens = 0
begin = time.time()


for i in range(len(bigtom_task1_ans)): # set to len(bigtom_task1_ans) later # 50 so far
    print(f'{i}-th story')
    for k in range(4):
        #story, event, protag_name, percept, initial_belief
        story = final_bigtom[i]['story']
        event = final_bigtom[i]['event']
        protag_name = final_bigtom[i]['protag_name']
        percept = final_bigtom[i]['percept_cue']
        initial_belief = final_bigtom[i]['initial_belief']
        prompt = bigtom_system.format(story = story, event = event, protag_name = protag_name, percept = percept, initial_belief = initial_belief)
        
        prompt += bigtom_q1[k].format(protag_name = protag_name, initial_belief = initial_belief)

        input_tokens += count_tokens(prompt)
        
        # print(prompt)
        # print("input tokens = ", count_tokens(prompt), ". price (USD) is ", count_tokens(prompt)*30/1000000)
        
        
        # Add back later
        output = LM_generation(prompt, temperature = 0, max_tokens = 3, model = model)
        llm_answer = output[0][0]
        bigtom_task1_ans[i]['llm_answer'][k] = llm_answer

                
        output_tokens += count_tokens(output[0])
        save_jsonl(bigtom_task1_ans, f'bigtom_task1_ans_{model}.jsonl')
        # print("output tokens = ", count_tokens(output[0]), ". price (USD) is ", count_tokens(output[0])*60/1000000)
                
    if i % 5 == 0:
        end = time.time()
        print(f'Time taken for these 5 stories: {end-begin} seconds.')
        begin = time.time()


print("input tokens = ", input_tokens, ". price (USD) is ", input_tokens*30/1000000)
print("output tokens = ", output_tokens, ". price (USD) is ", output_tokens*60/1000000)




0-th story


In [137]:
model = 'gpt-4o'
with open(f'bigtom_task1_ans_{model}.jsonl') as f: # ambiguous story
    bigtom_task1_ans = [json.loads(line) for line in f]

In [138]:
# Scoring
# Array: [story, groundtruth, sense] - correct & total

bigtom_task1_llm_answer = [instance['llm_answer'] for instance in bigtom_task1_ans]
bigtom_task1_true_answer = [np.array(['A', 'B'])[np.array(instance['true_answer'])] for instance in bigtom_task1_ans]
# bigtom_task1_llm_answer = bigtom_task1_llm_answer[0:50]
# bigtom_task1_true_answer = bigtom_task1_true_answer[0:50]
bigtom_task1_llm_answer = np.array(bigtom_task1_llm_answer)
bigtom_task1_true_answer = np.array(bigtom_task1_true_answer)
bigtom_task1_accuracy = (bigtom_task1_llm_answer == bigtom_task1_true_answer).mean(axis=1)
gt_id = {'positive':0, 'hard_negative': 1, 'easy_negative':2}
bigtom_task1_correct = np.zeros((100, 3, 8)) # groundtruth: pos, hard_neg, easy_neg
bigtom_task1_total = np.zeros((100, 3, 8))
for i, instance in enumerate(bigtom_task1_ans):
    ider = instance['ID']
    story_idx = int(ider.split('-')[0])
    gt_idx = gt_id[instance['groundtruth_classification']]
    sense_idx = sense_dict[instance['sense']]
    bigtom_task1_total[story_idx][gt_idx][sense_idx] = 1
    bigtom_task1_correct[story_idx][gt_idx][sense_idx] = bigtom_task1_accuracy[i]
eps = 1e-10
print("TPR, TNR-hard, TNR-easy: ", bigtom_task1_correct.sum(axis = 0).sum(axis = 1)/(bigtom_task1_total.sum(axis = 0).sum(axis = 1) + eps))


TPR, TNR-hard, TNR-easy:  [0.94592697 0.25195312 0.64458525]


In [336]:
# Supplementary - Template

with open('bigtom_filter.npy', 'rb') as f:
    bigtom_filter = np.load(f)
with open('opentom_filter.npy', 'rb') as f:
    opentom_filter = np.load(f)
    
from sklearn.metrics import classification_report
model = 'llama-405b'
with open(f'bigtom_task1_ans_{model}.jsonl') as f: # put story here
    task1_ans = [json.loads(line) for line in f]
#task1_ans = list(np.array(task1_ans)[opentom_filter]) # new
    


task1_llm_answer = [instance['llm_answer'] for instance in task1_ans]
task1_true_answer = [np.array(['A', 'B'])[np.array(instance['true_answer'])] for instance in task1_ans]
# bigtom_task1_llm_answer = bigtom_task1_llm_answer[0:50]
# bigtom_task1_true_answer = bigtom_task1_true_answer[0:50]
task1_llm_answer = np.array(task1_llm_answer)
task1_true_answer = np.array(task1_true_answer)

converted_llm_answer_hard = []
converted_true_answer_hard = []
converted_llm_answer_total = []
converted_true_answer_total = []
converted_llm_answer_easy_only = []
converted_true_answer_easy_only = []

# bigtom_task1_accuracy = (bigtom_task1_llm_answer == bigtom_task1_true_answer).mean(axis=1)
# gt_id = {'positive':0, 'hard_negative': 1, 'easy_negative':2}
# bigtom_task1_correct = np.zeros((100, 3, 8)) # groundtruth: pos, hard_neg, easy_neg
# bigtom_task1_total = np.zeros((100, 3, 8))
for idx, instance in enumerate(task1_ans):
    if instance['groundtruth_classification'] == 'positive':
        for i in range(len(instance['llm_answer'])):
            # insert to hard array
            # llm_answer = instance['llm_answer'][i]
            llm_correct = instance['llm_answer'][i] == np.array(['A', 'B'])[np.array(instance['true_answer'])][i]
            converted_true_answer_hard.append(1)
            converted_llm_answer_hard.append(int(llm_correct))
            converted_true_answer_total.append(1)
            converted_llm_answer_total.append(int(llm_correct))
            converted_true_answer_easy_only.append(1)
            converted_llm_answer_easy_only.append(int(llm_correct))
    else: # negative
        for i in range(len(instance['llm_answer'])):
            llm_correct = instance['llm_answer'][i] == np.array(['A', 'B'])[np.array(instance['true_answer'])][i]
            converted_true_answer_total.append(0)
            converted_llm_answer_total.append(1-int(llm_correct))
            if instance['groundtruth_classification'] == 'hard_negative':
                converted_true_answer_hard.append(0)
                converted_llm_answer_hard.append(1-int(llm_correct))
            else:
                converted_true_answer_easy_only.append(0)
                converted_llm_answer_easy_only.append(1-int(llm_correct))

print('positives (1) and hard negative (0):\n', classification_report(converted_true_answer_hard,converted_llm_answer_hard, digits = 3))
print('positives (1) and easy negative (0):\n', classification_report(converted_true_answer_easy_only,converted_llm_answer_easy_only, digits = 3))
print('positives (1) and all negative (0):\n', classification_report(converted_true_answer_total,converted_llm_answer_total, digits = 3))


positives (1) and hard negative (0):
               precision    recall  f1-score   support

           0      0.721     0.516     0.601      1092
           1      0.749     0.879     0.809      1796

    accuracy                          0.741      2888
   macro avg      0.735     0.697     0.705      2888
weighted avg      0.738     0.741     0.730      2888

positives (1) and easy negative (0):
               precision    recall  f1-score   support

           0      0.872     0.853     0.862      1736
           1      0.861     0.879     0.870      1796

    accuracy                          0.866      3532
   macro avg      0.866     0.866     0.866      3532
weighted avg      0.866     0.866     0.866      3532

positives (1) and all negative (0):
               precision    recall  f1-score   support

           0      0.904     0.723     0.803      2828
           1      0.668     0.879     0.759      1796

    accuracy                          0.783      4624
   macro avg   

In [276]:
# Supplementary

from sklearn.metrics import classification_report
# model = 'mistral'
# with open(f'bigtom_task1_ans_{model}.jsonl') as f: # ambiguous story
#     bigtom_task1_ans = [json.loads(line) for line in f]
labeller = {0:[0,0,0], 1:[1,0,0], 2:[1,1,0], 3:[1,1,1]}
task1_human_answer = [labeller[round(instance['belief_change_label_raw']*3)] for instance in final_bigtom]
task1_true_answer = [[instance['belief_change_label'], instance['belief_change_label'], instance['belief_change_label']] for instance in final_bigtom]
task1_human_answer = np.array(task1_human_answer)
task1_true_answer = np.array(task1_true_answer)

gt_class = np.array([instance['groundtruth_classification'] for instance in final_bigtom])


print('positives (1) and hard negative (0):\n', classification_report(task1_true_answer[gt_class != 'easy_negative'].reshape(-1),task1_human_answer[gt_class != 'easy_negative'].reshape(-1), digits = 3))
print('positives (1) and easy negative (0):\n', classification_report(task1_true_answer[gt_class != 'hard_negative'].reshape(-1),task1_human_answer[gt_class != 'hard_negative'].reshape(-1), digits = 3))
print('positives (1) and all negative (0):\n', classification_report(task1_true_answer.reshape(-1),task1_human_answer.reshape(-1), digits = 3))


positives (1) and hard negative (0):
               precision    recall  f1-score   support

           0      0.862     0.878     0.870       819
           1      0.925     0.915     0.920      1347

    accuracy                          0.901      2166
   macro avg      0.894     0.896     0.895      2166
weighted avg      0.901     0.901     0.901      2166

positives (1) and easy negative (0):
               precision    recall  f1-score   support

           0      0.918     0.991     0.953      1302
           1      0.990     0.915     0.951      1347

    accuracy                          0.952      2649
   macro avg      0.954     0.953     0.952      2649
weighted avg      0.955     0.952     0.952      2649

positives (1) and all negative (0):
               precision    recall  f1-score   support

           0      0.946     0.947     0.947      2121
           1      0.917     0.915     0.916      1347

    accuracy                          0.935      3468
   macro avg   

In [240]:
# BigToM first - Task 1
model = "gpt-35-turbo"
# positives
count = 0
input_tokens = 0
output_tokens = 0
begin = time.time()


for i in range(len(opentom_task1_ans)): # set to len(bigtom_task1_ans) later # 50 so far
    print(f'{i}-th story')
    for k in range(4):
        #story, event, protag_name, percept, initial_belief
        story = final_opentom[i]['story']
        protag_name = final_opentom[i]['protag_name']
        percept = final_opentom[i]['percept_cue']
        initial_belief = final_opentom[i]['initial_belief']
        prompt = opentom_system.format(story = story, protag_name = protag_name, percept = percept, initial_belief = initial_belief)
        
        prompt += opentom_q1[k].format(protag_name = protag_name, initial_belief = initial_belief)

        input_tokens += count_tokens(prompt)
        
        # print(prompt)
        # print("input tokens = ", count_tokens(prompt), ". price (USD) is ", count_tokens(prompt)*30/1000000)
        
        
        # Add back later
        output = LM_generation(prompt, temperature = 0, max_tokens = 3, model = model)
        llm_answer = output[0][0]
        opentom_task1_ans[i]['llm_answer'][k] = llm_answer

                
        output_tokens += count_tokens(output[0])
        save_jsonl(opentom_task1_ans, f'opentom_task1_ans_{model}.jsonl')
        # print("output tokens = ", count_tokens(output[0]), ". price (USD) is ", count_tokens(output[0])*60/1000000)
                
    if i % 5 == 0:
        end = time.time()
        print(f'Time taken for these 5 stories: {end-begin} seconds.')
        begin = time.time()


print("input tokens = ", input_tokens, ". price (USD) is ", input_tokens*30/1000000)
print("output tokens = ", output_tokens, ". price (USD) is ", output_tokens*60/1000000)




0-th story
Time taken for these 5 stories: 2.628728151321411 seconds.
1-th story
2-th story
3-th story
4-th story
5-th story
Time taken for these 5 stories: 9.53247880935669 seconds.
6-th story
7-th story
8-th story
9-th story
10-th story
Time taken for these 5 stories: 9.968948364257812 seconds.
11-th story
12-th story
13-th story
14-th story
15-th story
Time taken for these 5 stories: 41.69994807243347 seconds.
16-th story
17-th story
18-th story
19-th story
20-th story
Time taken for these 5 stories: 9.850733041763306 seconds.
21-th story
22-th story
23-th story
24-th story
25-th story
Time taken for these 5 stories: 9.28256893157959 seconds.
26-th story
27-th story
28-th story
29-th story
30-th story
Time taken for these 5 stories: 43.14979100227356 seconds.
31-th story
32-th story
33-th story
34-th story
35-th story
Time taken for these 5 stories: 10.294763803482056 seconds.
36-th story
37-th story
38-th story
39-th story
40-th story
Time taken for these 5 stories: 10.105647802352

In [241]:
# Scoring
# Array: [story, groundtruth, sense] - correct & total

opentom_task1_llm_answer = [instance['llm_answer'] for instance in opentom_task1_ans]
opentom_task1_true_answer = [np.array(['A', 'B'])[np.array(instance['true_answer'])] for instance in opentom_task1_ans]
opentom_task1_llm_answer = np.array(opentom_task1_llm_answer)
opentom_task1_true_answer = np.array(opentom_task1_true_answer)
opentom_task1_accuracy = (opentom_task1_llm_answer == opentom_task1_true_answer).mean(axis=1)
gt_id = {'positive':0, 'hard_negative': 1, 'easy_negative':2}
opentom_task1_correct = np.zeros((121, 3, 8)) # groundtruth: pos, hard_neg, easy_neg
opentom_task1_total = np.zeros((121, 3, 8))
for i, instance in enumerate(opentom_task1_ans):
    ider = instance['ID']
    story_idx = int(ider.split('-')[0])
    gt_idx = gt_id[instance['groundtruth_classification']]
    sense_idx = sense_dict[instance['sense']]
    opentom_task1_total[story_idx][gt_idx][sense_idx] = 1
    opentom_task1_correct[story_idx][gt_idx][sense_idx] = opentom_task1_accuracy[i]
    

In [242]:
eps = 1e-10
print("TPR, TNR-hard, TNR-easy: ", opentom_task1_correct.sum(axis = 0).sum(axis = 1)/(opentom_task1_total.sum(axis = 0).sum(axis = 1) + eps))

TPR, TNR-hard, TNR-easy:  [0.92040094 0.08248731 0.20432692]


Task 2

In [243]:
# BigToM first - Task 1
model = "gpt-35-turbo"
# positives
count = 0
input_tokens = 0
output_tokens = 0
begin = time.time()


for i in range(len(bigtom_task2_ans)): # set to len(bigtom_task1_ans) later # 50 so far
    print(f'{i}-th story')
    for k in range(5):
        #story, event, protag_name, percept, initial_belief
        story = final_bigtom[i]['story']
        event = final_bigtom[i]['event']
        protag_name = final_bigtom[i]['protag_name']
        percept = final_bigtom[i]['percept_cue']
        initial_belief = final_bigtom[i]['initial_belief']
        prompt = bigtom_system.format(story = story, event = event, protag_name = protag_name, percept = percept, initial_belief = initial_belief)
        
        default_question = final_bigtom[i]['default_question']
        randomizer = np.array(bigtom_task2_ans[i]['answer_order'][k])
        # print(final_bigtom[i]['answer_options'])
        # print(randomizer)
        first = final_bigtom[i]['answer_options'][randomizer[0]]
        second = final_bigtom[i]['answer_options'][randomizer[1]]
        third = final_bigtom[i]['answer_options'][randomizer[2]]
        fourth = final_bigtom[i]['answer_options'][randomizer[3]]
        prompt += bigtom_q2_prompt.format(default_question=default_question, first=first, second=second, third=third, fourth=fourth)

        input_tokens += count_tokens(prompt)
        
        # print(prompt)
        # print("input tokens = ", count_tokens(prompt), ". price (USD) is ", count_tokens(prompt)*30/1000000)
        
        
        # Add back later
        output = LM_generation(prompt, temperature = 0, max_tokens = 3, model = model)
        llm_answer = output[0][0]
        bigtom_task2_ans[i]['llm_answer'][k] = llm_answer

                
        output_tokens += count_tokens(output[0])
        save_jsonl(bigtom_task2_ans, f'bigtom_task2_ans_{model}.jsonl')
        # print("output tokens = ", count_tokens(output[0]), ". price (USD) is ", count_tokens(output[0])*60/1000000)
                
    if i % 5 == 0:
        end = time.time()
        print(f'Time taken for these 5 stories: {end-begin} seconds.')
        begin = time.time()


print("input tokens = ", input_tokens, ". price (USD) is ", input_tokens*30/1000000)
print("output tokens = ", output_tokens, ". price (USD) is ", output_tokens*60/1000000)




0-th story
Time taken for these 5 stories: 3.575814962387085 seconds.
1-th story
2-th story
3-th story
4-th story
5-th story
Time taken for these 5 stories: 13.853145122528076 seconds.
6-th story
7-th story
8-th story
9-th story
10-th story
Time taken for these 5 stories: 12.758174896240234 seconds.
11-th story
12-th story
13-th story
14-th story
15-th story
Time taken for these 5 stories: 44.17573308944702 seconds.
16-th story
17-th story
18-th story
19-th story
20-th story
Time taken for these 5 stories: 12.66422414779663 seconds.
21-th story
22-th story
23-th story
24-th story
25-th story
Time taken for these 5 stories: 40.650043964385986 seconds.
26-th story
27-th story
28-th story
29-th story
30-th story
Time taken for these 5 stories: 15.46507477760315 seconds.
31-th story
32-th story
33-th story
34-th story
35-th story
Time taken for these 5 stories: 13.438211917877197 seconds.
36-th story
37-th story
38-th story
39-th story
40-th story
Time taken for these 5 stories: 45.4395089

In [244]:
bigtom_task2_ans

[{'ID': '0-0-0',
  'groundtruth_classification': 'positive',
  'sense': 'visual',
  'answer_order': [[2, 3, 1, 0],
   [0, 2, 1, 3],
   [3, 0, 2, 1],
   [1, 0, 2, 3],
   [1, 2, 3, 0]],
  'llm_answer': ['C', 'C', 'A', 'A', 'A'],
  'true_answer': ['B', 'D', 'A', 'D', 'C'],
  'true_answer_og': 3},
 {'ID': '0-0-1',
  'groundtruth_classification': 'hard_negative',
  'sense': 'visual',
  'answer_order': [[0, 3, 2, 1],
   [1, 0, 2, 3],
   [0, 2, 3, 1],
   [2, 3, 0, 1],
   [2, 1, 3, 0]],
  'llm_answer': ['D', 'A', 'D', 'D', 'B'],
  'true_answer': ['A', 'B', 'A', 'C', 'D'],
  'true_answer_og': 0},
 {'ID': '0-0-2',
  'groundtruth_classification': 'positive',
  'sense': 'visual',
  'answer_order': [[0, 3, 2, 1],
   [1, 2, 0, 3],
   [2, 1, 0, 3],
   [0, 2, 1, 3],
   [0, 1, 2, 3]],
  'llm_answer': ['B', 'A', 'D', 'C', 'B'],
  'true_answer': ['D', 'A', 'B', 'C', 'B'],
  'true_answer_og': 1},
 {'ID': '0-0-3',
  'groundtruth_classification': 'positive',
  'sense': 'auditory',
  'answer_order': [[3, 2, 

In [45]:
model = 'llama-8b'
with open(f'bigtom_task2_ans_{model}.jsonl') as f: # ambiguous story
    bigtom_task2_ans = [json.loads(line) for line in f]

In [46]:
bigtom_task2_llm_answer = [instance['llm_answer'][:3] for instance in bigtom_task2_ans]
bigtom_task2_true_answer = [instance['true_answer'][:3] for instance in bigtom_task2_ans]
# bigtom_task2_llm_answer = bigtom_task2_llm_answer[0:50]
# bigtom_task2_true_answer = bigtom_task2_true_answer[0:50]
bigtom_task2_llm_answer = np.array(bigtom_task2_llm_answer)
bigtom_task2_true_answer = np.array(bigtom_task2_true_answer)
bigtom_task2_accuracy = (bigtom_task2_llm_answer == bigtom_task2_true_answer).mean(axis=1)

In [47]:
gt_id = {'positive':0, 'hard_negative': 1, 'easy_negative':2}
bigtom_task2_correct = np.zeros((100, 3, 8)) # groundtruth: pos, hard_neg, easy_neg
bigtom_task2_total = np.zeros((100, 3, 8))
for i, instance in enumerate(bigtom_task2_ans):
    ider = instance['ID']
    story_idx = int(ider.split('-')[0])
    gt_idx = gt_id[instance['groundtruth_classification']]
    sense_idx = sense_dict[instance['sense']]
    bigtom_task2_total[story_idx][gt_idx][sense_idx] = 1
    bigtom_task2_correct[story_idx][gt_idx][sense_idx] = bigtom_task2_accuracy[i]
    

In [48]:
eps = 1e-10
print("TPR, TNR-hard, TNR-easy: ", bigtom_task2_correct.sum(axis = 0).sum(axis = 1)/(bigtom_task2_total.sum(axis = 0).sum(axis = 1) + eps))

TPR, TNR-hard, TNR-easy:  [0.70131086 0.01692708 0.02304147]


In [178]:
task2_ans[0]

{'ID': '0-0-0',
 'groundtruth_classification': 'positive',
 'sense': 'visual',
 'answer_order': [[2, 0, 3, 1],
  [3, 2, 0, 1],
  [1, 3, 0, 2],
  [0, 2, 1, 3],
  [2, 3, 0, 1]],
 'llm_answer': ['D', 'D', 'A', 'C', 'D'],
 'true_answer': ['D', 'D', 'A', 'C', 'D'],
 'true_answer_og': 1}

In [ ]:
# Supplementary - Template
import random
import json
import numpy as np

random.seed(0)

from sklearn.metrics import classification_report, precision_recall_fscore_support
model = 'deepseek-reasoner'
with open(f'opentom_task2_ans_{model}.jsonl') as f: # put story here
    task2_ans = [json.loads(line) for line in f]

# task2_ans = list(np.array(task2_ans)[bigtom_filter]) # new
    
letters = {'A':0, 'B':1, 'C':2, 'D':3}

task2_llm_answer = [instance['llm_answer'][:3] for instance in task2_ans]
task2_true_answer = [instance['true_answer'][:3] for instance in task2_ans]
# bigtom_task1_llm_answer = bigtom_task1_llm_answer[0:50]
# bigtom_task1_true_answer = bigtom_task1_true_answer[0:50]
task2_llm_answer = np.array(task2_llm_answer)
task2_true_answer = np.array(task2_true_answer)

converted_llm_answer_hard = []
converted_true_answer_hard = []
converted_llm_answer_total = []
converted_true_answer_total = []
converted_llm_answer_easy_only = []
converted_true_answer_easy_only = []

# bigtom_task1_accuracy = (bigtom_task1_llm_answer == bigtom_task1_true_answer).mean(axis=1)
# gt_id = {'positive':0, 'hard_negative': 1, 'easy_negative':2}
# bigtom_task1_correct = np.zeros((100, 3, 8)) # groundtruth: pos, hard_neg, easy_neg
# bigtom_task1_total = np.zeros((100, 3, 8))
for idx, instance in enumerate(task2_ans):
    for i in range(3):
        true_answer = instance['true_answer_og']
        llm_answer_index = instance['llm_answer'][i]
        if llm_answer_index not in ['A','B','C','D']:
            llm_answer_index = random.choice(['A','B','C','D'])
        llm_answer_index = letters[llm_answer_index]
        llm_answer = instance['answer_order'][i][llm_answer_index]
        converted_true_answer_total.append(true_answer)
        converted_llm_answer_total.append(llm_answer)
        if instance['groundtruth_classification'] == 'positive':
            converted_true_answer_hard.append(true_answer)
            converted_llm_answer_hard.append(llm_answer)
            converted_true_answer_easy_only.append(true_answer)
            converted_llm_answer_easy_only.append(llm_answer)
        else: # negative
            if instance['groundtruth_classification'] == 'hard_negative':
                converted_true_answer_hard.append(true_answer)
                converted_llm_answer_hard.append(llm_answer)
            else:
                converted_true_answer_easy_only.append(true_answer)
                converted_llm_answer_easy_only.append(llm_answer)

print('positives (1) and hard negative (0):\n', classification_report(converted_true_answer_hard,converted_llm_answer_hard, digits = 3,labels=[0,1,2,3]))
print(precision_recall_fscore_support(converted_true_answer_hard, converted_llm_answer_hard, average = 'micro', labels=[0,1,2,3]))
print('positives (1) and easy negative (0):\n', classification_report(converted_true_answer_easy_only,converted_llm_answer_easy_only, digits = 3,labels=[0,1,2,3]))
print(precision_recall_fscore_support(converted_true_answer_easy_only, converted_llm_answer_easy_only, average = 'micro', labels=[0,1,2,3]))
print('positives (1) and all negative (0):\n', classification_report(converted_true_answer_total,converted_llm_answer_total, digits = 3,labels=[0,1,2,3]))
print(precision_recall_fscore_support(converted_true_answer_total, converted_llm_answer_total, average = 'micro', labels=[0,1,2,3]))


positives (1) and hard negative (0):
               precision    recall  f1-score   support

           0      0.636     0.452     0.528       651
           1      0.822     0.870     0.845      1767
           2      0.000     0.000     0.000         0
           3      0.000     0.000     0.000         0

    accuracy                          0.757      2418
   macro avg      0.365     0.330     0.343      2418
weighted avg      0.772     0.757     0.760      2418

(0.7572373862696443, 0.7572373862696443, 0.7572373862696442, None)
positives (1) and easy negative (0):
               precision    recall  f1-score   support

           0      0.867     0.701     0.775      1560
           1      0.774     0.870     0.819      1767
           2      0.000     0.000     0.000         0
           3      0.000     0.000     0.000         0

    accuracy                          0.791      3327
   macro avg      0.410     0.393     0.399      3327
weighted avg      0.817     0.791     0.79

/Users/dkoffical/opt/anaconda3/envs/COMP4222/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/dkoffical/opt/anaconda3/envs/COMP4222/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/dkoffical/opt/anaconda3/envs/COMP4222/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

In [199]:
# final_opentom

In [297]:
# Supplementary

from sklearn.metrics import classification_report
# model = 'mistral'
# with open(f'bigtom_task1_ans_{model}.jsonl') as f: # ambiguous story
#     bigtom_task1_ans = [json.loads(line) for line in f]
task1_human_answer = [instance['new_belief_label_raw'] for instance in final_bigtom]
task1_true_answer = [[instance['new_belief_label'], instance['new_belief_label'], instance['new_belief_label']] for instance in final_bigtom]
task1_human_answer = np.array(task1_human_answer)
task1_true_answer = np.array(task1_true_answer)

gt_class = np.array([instance['groundtruth_classification'] for instance in final_bigtom])


print('positives (1) and hard negative (0):\n', classification_report(task1_true_answer[gt_class != 'easy_negative'].reshape(-1),task1_human_answer[gt_class != 'easy_negative'].reshape(-1), digits = 3, labels =['A','B','C','D']))
print('positives (1) and easy negative (0):\n', classification_report(task1_true_answer[gt_class != 'hard_negative'].reshape(-1),task1_human_answer[gt_class != 'hard_negative'].reshape(-1), digits = 3, labels = ['A','B','C','D']))
print('positives (1) and all negative (0):\n', classification_report(task1_true_answer.reshape(-1),task1_human_answer.reshape(-1), digits = 3,  labels =['A','B','C','D']))


positives (1) and hard negative (0):
               precision    recall  f1-score   support

           A      0.862     0.878     0.870       819
           B      0.931     0.884     0.907      1290
           C      0.429     0.667     0.522        27
           D      0.328     0.700     0.447        30

    accuracy                          0.877      2166
   macro avg      0.637     0.782     0.686      2166
weighted avg      0.890     0.877     0.882      2166

positives (1) and easy negative (0):
               precision    recall  f1-score   support

           A      0.918     0.991     0.953      1302
           B      0.979     0.884     0.930      1290
           C      0.486     0.667     0.562        27
           D      0.500     0.700     0.583        30

    accuracy                          0.932      2649
   macro avg      0.721     0.810     0.757      2649
weighted avg      0.939     0.932     0.933      2649

positives (1) and all negative (0):
               pre

In [ ]:
# from sklearn.metrics import classification_report
# target_names = 

In [ ]:
# # save_jsonl(opentom_task2_ans, f'opentom_task2_ans_{model}.jsonl')

# with open('opentom_task2_ans_gpt-4o.jsonl') as f: # ambiguous story
#     opentom_task2_ans = [json.loads(line) for line in f]

In [260]:
# OpenToM first - Task 1
model = "gpt-35-turbo"
# positives
count = 0
input_tokens = 0
output_tokens = 0
begin = time.time()

# for i in range(2):
for i in range(len(opentom_task2_ans)): # set to len(bigtom_task1_ans) later # 50 so far
    print(f'{i}-th story')
    for k in range(5):
        #story, event, protag_name, percept, initial_belief
        story = final_opentom[i]['story']
        protag_name = final_opentom[i]['protag_name']
        percept = final_opentom[i]['percept_cue']
        initial_belief = final_opentom[i]['initial_belief']
        prompt = opentom_system.format(story = story, protag_name = protag_name, percept = percept, initial_belief = initial_belief)
        
        default_question = final_opentom[i]['default_question']
        randomizer = np.array(opentom_task2_ans[i]['answer_order'][k])
        # print(final_bigtom[i]['answer_options'])
        # print(randomizer)
        first = final_opentom[i]['answer_options'][randomizer[0]]
        second = final_opentom[i]['answer_options'][randomizer[1]]
        third = final_opentom[i]['answer_options'][randomizer[2]]
        fourth = final_opentom[i]['answer_options'][randomizer[3]]
        prompt += opentom_q2_prompt.format(default_question=default_question, first=first, second=second, third=third, fourth=fourth)

        input_tokens += count_tokens(prompt)
        
        # print(prompt)
        # print("input tokens = ", count_tokens(prompt), ". price (USD) is ", count_tokens(prompt)*30/1000000)
        
        
        # Add back later
        output = LM_generation(prompt, temperature = 0, max_tokens = 3, model = model)
        llm_answer = output[0][0]
        opentom_task2_ans[i]['llm_answer'][k] = llm_answer

                
        output_tokens += count_tokens(output[0])
        save_jsonl(opentom_task2_ans, f'opentom_task2_ans_{model}.jsonl')
        # print("output tokens = ", count_tokens(output[0]), ". price (USD) is ", count_tokens(output[0])*60/1000000)
                
    if i % 5 == 0:
        end = time.time()
        print(f'Time taken for these 5 stories: {end-begin} seconds.')
        begin = time.time()


print("input tokens = ", input_tokens, ". price (USD) is ", input_tokens*30/1000000)
print("output tokens = ", output_tokens, ". price (USD) is ", output_tokens*60/1000000)




1156-th story
1157-th story
1158-th story
1159-th story
1160-th story
Time taken for these 5 stories: 15.131700992584229 seconds.
1161-th story
1162-th story
1163-th story
1164-th story
1165-th story
Time taken for these 5 stories: 15.193772792816162 seconds.
1166-th story
1167-th story
1168-th story
1169-th story
1170-th story
Time taken for these 5 stories: 39.66163206100464 seconds.
1171-th story
1172-th story
1173-th story
1174-th story
1175-th story
Time taken for these 5 stories: 19.622599124908447 seconds.
1176-th story
1177-th story
1178-th story
1179-th story
1180-th story
Time taken for these 5 stories: 38.56685495376587 seconds.
1181-th story
1182-th story
1183-th story
1184-th story
1185-th story
Time taken for these 5 stories: 14.399535179138184 seconds.
1186-th story
1187-th story
1188-th story
1189-th story
1190-th story
Time taken for these 5 stories: 18.002933740615845 seconds.
1191-th story
1192-th story
1193-th story
1194-th story
1195-th story
Time taken for these 5

In [58]:
model = 'gpt-35-turbo'
with open(f'opentom_task2_ans_{model}.jsonl') as f: # ambiguous story
    opentom_task2_ans = [json.loads(line) for line in f]

In [59]:
opentom_task2_llm_answer = [instance['llm_answer'][:3] for instance in opentom_task2_ans]
opentom_task2_true_answer = [instance['true_answer'][:3] for instance in opentom_task2_ans]
opentom_task2_llm_answer = np.array(opentom_task2_llm_answer)
opentom_task2_true_answer = np.array(opentom_task2_true_answer)
opentom_task2_accuracy = (opentom_task2_llm_answer == opentom_task2_true_answer).mean(axis=1)
gt_id = {'positive':0, 'hard_negative': 1, 'easy_negative':2}
opentom_task2_correct = np.zeros((121, 3, 8)) # groundtruth: pos, hard_neg, easy_neg
opentom_task2_total = np.zeros((121, 3, 8))
for i, instance in enumerate(opentom_task2_ans):
    ider = instance['ID']
    story_idx = int(ider.split('-')[0])
    gt_idx = gt_id[instance['groundtruth_classification']]
    sense_idx = sense_dict[instance['sense']]
    opentom_task2_total[story_idx][gt_idx][sense_idx] = 1
    opentom_task2_correct[story_idx][gt_idx][sense_idx] = opentom_task2_accuracy[i]
eps = 1e-10
print("TPR, TNR-hard, TNR-easy: ", opentom_task2_correct.sum(axis = 0).sum(axis = 1)/(opentom_task2_total.sum(axis = 0).sum(axis = 1) + eps))

TPR, TNR-hard, TNR-easy:  [0.97562893 0.11167513 0.07051282]


Task 3

In [262]:
final_bigtom_with_noise = []
bigtom_noise_percepts = {i:[] for i in range(100)}
final_opentom_with_noise = []
opentom_noise_percepts = {i:[] for i in range(121)}
random.seed(0)
for instance in final_bigtom:
    if instance['groundtruth_classification'] == 'easy_negative':
        # find ID first
        story_idx = instance['ID'].split('-')[0]
        story_idx = int(story_idx)
        bigtom_noise_percepts[story_idx].append(instance['percept_cue'])
    else:
        final_bigtom_with_noise.append(instance)
# print(bigtom_noise_percepts[0])

rem_list = []
for instance in final_bigtom_with_noise:
    story_idx = instance['ID'].split('-')[0]
    story_idx = int(story_idx)
    # put the noise into the dataset
    instance['noise'] = []
    # if (len(bigtom_noise_percepts[story_idx])==0):
    #     print(instance)
    # if story_idx == 91:
    #     print(instance['ID'])
    for _ in range(5):
        if len(bigtom_noise_percepts[story_idx]) == 0:
            # print(instance['ID'])
            # final_bigtom_with_noise.remove(instance)
            rem_list.append(instance)
            break
        else:
            # print((bigtom_noise_percepts[story_idx]))
            instance['noise'].append(random.choice(bigtom_noise_percepts[story_idx]))
for story_idx in range(100):
    if len(bigtom_noise_percepts[story_idx]) == 0:
        del bigtom_noise_percepts[story_idx]
for instance in rem_list:
    final_bigtom_with_noise.remove(instance)
            
for instance in final_opentom:
    if instance['groundtruth_classification'] == 'easy_negative':
        # find ID first
        story_idx = instance['ID'].split('-')[0]
        story_idx = int(story_idx)
        opentom_noise_percepts[story_idx].append(instance['percept_cue'])
    else:
        final_opentom_with_noise.append(instance)
# print(bigtom_noise_percepts[0])
rem_list = []
for instance in final_opentom_with_noise:
    story_idx = instance['ID'].split('-')[0]
    story_idx = int(story_idx)
    # put the noise into the dataset
    instance['noise'] = []
    # if (len(bigtom_noise_percepts[story_idx])==0):
    #     print(instance)
    for _ in range(5):
        if len(opentom_noise_percepts[story_idx]) == 0:
            # final_opentom_with_noise.remove(instance)
            rem_list.append(instance)
            break
        else:
            # print((bigtom_noise_percepts[story_idx]))
            instance['noise'].append(random.choice(opentom_noise_percepts[story_idx]))
            
for story_idx in range(121):
    if len(opentom_noise_percepts[story_idx]) == 0:
        del opentom_noise_percepts[story_idx]
for instance in rem_list:
    final_bigtom_with_noise.remove(instance)
    
# No need to run again
import json
with open("llm_ready_bigtom_with_noise.jsonl", "w") as file:
    for item in final_bigtom_with_noise:
        json_string = json.dumps(item)
        file.write(json_string + '\n')

with open("llm_ready_opentom_with_noise.jsonl", "w") as file:
    for item in final_opentom_with_noise:
        json_string = json.dumps(item)
        file.write(json_string + '\n')
        
bigtom_task3_ans = [{'ID': instance['ID'], 'groundtruth_classification': instance['groundtruth_classification']
                     , 'sense': instance['sense'],
                     'answer_order': [[0,1], [1,0],[0,1],[1,0], [0,1], [1,0],[0,1],[1,0]], # this is the f() that can map original order to postprocess order
                     'llm_answer': ['X','X','X','X','X','X','X','X'],
                     'true_answer': [instance['belief_change_label'], 1 - instance['belief_change_label'], instance['belief_change_label'], 1 - instance['belief_change_label'], instance['belief_change_label'], 1 - instance['belief_change_label'], instance['belief_change_label'], 1 - instance['belief_change_label']]} 
                    for instance in final_bigtom_with_noise]
opentom_task3_ans = [{'ID': instance['ID'], 'groundtruth_classification': instance['groundtruth_classification']
                     , 'sense': instance['sense'],
                     'answer_order': [[0,1], [1,0],[0,1],[1,0], [0,1], [1,0],[0,1],[1,0]], # this is the f() that can map original order to postprocess order
                     'llm_answer': ['X','X','X','X','X','X','X','X'],
                     'true_answer': [instance['belief_change_label'], 1 - instance['belief_change_label'], instance['belief_change_label'], 1 - instance['belief_change_label'], instance['belief_change_label'], 1 - instance['belief_change_label'], instance['belief_change_label'], 1 - instance['belief_change_label']]} 
                    for instance in final_opentom_with_noise]


In [169]:
# save_jsonl(bigtom_task3_ans, f'bigtom_task3_ans_{model}.jsonl')

In [264]:
# BigToM first - Task 3
model = "gpt-35-turbo"
# positives
count = 0
input_tokens = 0
output_tokens = 0
begin = time.time()

# for i in range(11,50):
for i in range(len(bigtom_task3_ans)): # set to len(bigtom_task3_ans) later # 50 so far
    print(f'{i}-th story')
    for k in range(4):
        #story, event, protag_name, percept, initial_belief
        for j in range(2):# percept_order
            story = final_bigtom_with_noise[i]['story']
            event = final_bigtom_with_noise[i]['event']
            protag_name = final_bigtom_with_noise[i]['protag_name']
            percept_signal = final_bigtom_with_noise[i]['percept_cue']
            # in each iteration, pick a different noise
            percept_noise = final_bigtom_with_noise[i]['noise'][k]
            
            if (j==0):
                percept = f"\n1. {percept_signal} \n2. {percept_noise}\n"
            else:
                percept = f"\n1. {percept_noise} \n2. {percept_signal}\n"
            
            
            initial_belief = final_bigtom_with_noise[i]['initial_belief']
            prompt = bigtom_system.format(story = story, event = event, protag_name = protag_name, percept = percept, initial_belief = initial_belief)
            
            prompt += bigtom_q1[k].format(protag_name = protag_name, initial_belief = initial_belief)

            input_tokens += count_tokens(prompt)
            
            # print(prompt)
            # print("input tokens = ", count_tokens(prompt), ". price (USD) is ", count_tokens(prompt)*30/1000000)
            
            
            # Add back later
            output = LM_generation(prompt, temperature = 0, max_tokens = 3, model = model)
            llm_answer = output[0][0]
            bigtom_task3_ans[i]['llm_answer'][k + 4*j] = llm_answer

                    
            output_tokens += count_tokens(output[0])
            save_jsonl(bigtom_task3_ans, f'bigtom_task3_ans_{model}.jsonl')
            # print("output tokens = ", count_tokens(output[0]), ". price (USD) is ", count_tokens(output[0])*60/1000000)
                    
    if i % 5 == 0:
        end = time.time()
        print(f'Time taken for these 5 stories: {end-begin} seconds.')
        begin = time.time()


print("input tokens = ", input_tokens, ". price (USD) is ", input_tokens*30/1000000)
print("output tokens = ", output_tokens, ". price (USD) is ", output_tokens*60/1000000)




0-th story
Time taken for these 5 stories: 5.615377902984619 seconds.
1-th story
2-th story
3-th story
4-th story
5-th story
Time taken for these 5 stories: 19.109182119369507 seconds.
6-th story
7-th story
8-th story
9-th story
10-th story
Time taken for these 5 stories: 52.856900215148926 seconds.
11-th story
12-th story
13-th story
14-th story
15-th story
Time taken for these 5 stories: 51.635985136032104 seconds.
16-th story
17-th story
18-th story
19-th story
20-th story
Time taken for these 5 stories: 20.555409908294678 seconds.
21-th story
22-th story
23-th story
24-th story
25-th story
Time taken for these 5 stories: 48.99764919281006 seconds.
26-th story
27-th story
28-th story
29-th story
30-th story
Time taken for these 5 stories: 51.866185426712036 seconds.
31-th story
32-th story
33-th story
34-th story
35-th story
Time taken for these 5 stories: 24.474447011947632 seconds.
36-th story
37-th story
38-th story
39-th story
40-th story
Time taken for these 5 stories: 51.98158

In [ ]:
# with open('bigtom_task3_ans_gpt-4o.jsonl') as f: # ambiguous story
#     bigtom_task3_ans = [json.loads(line) for line in f]

In [265]:

bigtom_task3_llm_answer = [instance['llm_answer'] for instance in bigtom_task3_ans]
bigtom_task3_true_answer = [np.array(['A', 'B'])[np.array(instance['true_answer'])] for instance in bigtom_task3_ans]
# bigtom_task3_llm_answer = bigtom_task3_llm_answer[0:50]
# bigtom_task3_true_answer = bigtom_task3_true_answer[0:50]
bigtom_task3_llm_answer = np.array(bigtom_task3_llm_answer)
bigtom_task3_true_answer = np.array(bigtom_task3_true_answer)
bigtom_task3_accuracy = (bigtom_task3_llm_answer == bigtom_task3_true_answer).mean(axis=1)
gt_id = {'positive':0, 'hard_negative': 1}
bigtom_task3_correct = np.zeros((100, 2, 8)) # groundtruth: pos, hard_neg, easy_neg
bigtom_task3_total = np.zeros((100, 2, 8))
for i, instance in enumerate(bigtom_task3_ans):
    ider = instance['ID']
    story_idx = int(ider.split('-')[0])
    gt_idx = gt_id[instance['groundtruth_classification']]
    sense_idx = sense_dict[instance['sense']]
    bigtom_task3_total[story_idx][gt_idx][sense_idx] = 1
    bigtom_task3_correct[story_idx][gt_idx][sense_idx] = bigtom_task3_accuracy[i]
eps = 1e-10
print("TPR, TNR-hard: ", bigtom_task3_correct.sum(axis = 0).sum(axis = 1)/(bigtom_task3_total.sum(axis = 0).sum(axis = 1) + eps))

TPR, TNR-hard:  [0.99679487 0.00585937]


In [266]:
# BigToM first - Task 3
model = "gpt-35-turbo"
# positives
count = 0
input_tokens = 0
output_tokens = 0
begin = time.time()

# for i in range(2):
for i in range(len(opentom_task3_ans)): # set to len(bigtom_task3_ans) later # 50 so far
    print(f'{i}-th story')
    for k in range(4):
        #story, event, protag_name, percept, initial_belief
        for j in range(2):# percept_order
            story = final_opentom_with_noise[i]['story']
            protag_name = final_opentom_with_noise[i]['protag_name']
            percept_signal = final_opentom_with_noise[i]['percept_cue']
            # in each iteration, pick a different noise
            percept_noise = final_opentom_with_noise[i]['noise'][k]
            
            if (j==0):
                percept = f"\n1. {percept_signal} \n2. {percept_noise}\n"
            else:
                percept = f"\n1. {percept_noise} \n2. {percept_signal}\n"
            
            
            initial_belief = final_opentom_with_noise[i]['initial_belief']
            prompt = opentom_system.format(story = story, protag_name = protag_name, percept = percept, initial_belief = initial_belief)
            
            prompt += opentom_q1[k].format(protag_name = protag_name, initial_belief = initial_belief)

            input_tokens += count_tokens(prompt)
            
            # print(prompt)
            # print("input tokens = ", count_tokens(prompt), ". price (USD) is ", count_tokens(prompt)*30/1000000)
            
            
            # Add back later
            output = LM_generation(prompt, temperature = 0, max_tokens = 3, model = model)
            llm_answer = output[0][0]
            opentom_task3_ans[i]['llm_answer'][k + 4*j] = llm_answer

                    
            output_tokens += count_tokens(output[0])
            save_jsonl(opentom_task3_ans, f'opentom_task3_ans_{model}.jsonl')
            # print("output tokens = ", count_tokens(output[0]), ". price (USD) is ", count_tokens(output[0])*60/1000000)
                    
    if i % 5 == 0:
        end = time.time()
        print(f'Time taken for these 5 stories: {end-begin} seconds.')
        begin = time.time()


print("input tokens = ", input_tokens, ". price (USD) is ", input_tokens*30/1000000)
print("output tokens = ", output_tokens, ". price (USD) is ", output_tokens*60/1000000)




0-th story
Time taken for these 5 stories: 4.2326130867004395 seconds.
1-th story
2-th story
3-th story
4-th story
5-th story
Time taken for these 5 stories: 21.62366223335266 seconds.
6-th story
7-th story
8-th story
9-th story
10-th story
Time taken for these 5 stories: 49.728187084198 seconds.
11-th story
12-th story
13-th story
14-th story
15-th story
Time taken for these 5 stories: 51.85366129875183 seconds.
16-th story
17-th story
18-th story
19-th story
20-th story
Time taken for these 5 stories: 26.001206874847412 seconds.
21-th story
22-th story
23-th story
24-th story
25-th story
Time taken for these 5 stories: 44.1051230430603 seconds.
26-th story
27-th story
28-th story
29-th story
30-th story
Time taken for these 5 stories: 53.999122858047485 seconds.
31-th story
32-th story
33-th story
34-th story
35-th story
Time taken for these 5 stories: 24.099774837493896 seconds.
36-th story
37-th story
38-th story
39-th story
40-th story
Time taken for these 5 stories: 47.7096140384

In [267]:

opentom_task3_llm_answer = [instance['llm_answer'] for instance in opentom_task3_ans]
opentom_task3_true_answer = [np.array(['A', 'B'])[np.array(instance['true_answer'])] for instance in opentom_task3_ans]
# bigtom_task3_llm_answer = bigtom_task3_llm_answer[0:50]
# bigtom_task3_true_answer = bigtom_task3_true_answer[0:50]
opentom_task3_llm_answer = np.array(opentom_task3_llm_answer)
opentom_task3_true_answer = np.array(opentom_task3_true_answer)
opentom_task3_accuracy = (opentom_task3_llm_answer == opentom_task3_true_answer).mean(axis=1)
gt_id = {'positive':0, 'hard_negative': 1}
opentom_task3_correct = np.zeros((121, 2, 8)) # groundtruth: pos, hard_neg, easy_neg
opentom_task3_total = np.zeros((121, 2, 8))
for i, instance in enumerate(opentom_task3_ans):
    ider = instance['ID']
    story_idx = int(ider.split('-')[0])
    gt_idx = gt_id[instance['groundtruth_classification']]
    sense_idx = sense_dict[instance['sense']]
    opentom_task3_total[story_idx][gt_idx][sense_idx] = 1
    opentom_task3_correct[story_idx][gt_idx][sense_idx] = opentom_task3_accuracy[i]
eps = 1e-10
print("TPR, TNR-hard: ", opentom_task3_correct.sum(axis = 0).sum(axis = 1)/(opentom_task3_total.sum(axis = 0).sum(axis = 1) + eps))

TPR, TNR-hard:  [0.89887972 0.10659898]


In [6]:
# Supplementary - Template

from sklearn.metrics import classification_report
model = 'deepseek-reasoner'
with open(f'opentom_task3_ans_{model}.jsonl') as f: # put story here
    task3_ans = [json.loads(line) for line in f]

task3_llm_answer = [instance['llm_answer'] for instance in task3_ans]
task3_true_answer = [np.array(['A', 'B'])[np.array(instance['true_answer'])] for instance in task3_ans]
# bigtom_task1_llm_answer = bigtom_task1_llm_answer[0:50]
# bigtom_task1_true_answer = bigtom_task1_true_answer[0:50]
task3_llm_answer = np.array(task3_llm_answer)
task3_true_answer = np.array(task3_true_answer)

converted_llm_answer_total = []
converted_true_answer_total = []

# bigtom_task1_accuracy = (bigtom_task1_llm_answer == bigtom_task1_true_answer).mean(axis=1)
# gt_id = {'positive':0, 'hard_negative': 1, 'easy_negative':2}
# bigtom_task1_correct = np.zeros((100, 3, 8)) # groundtruth: pos, hard_neg, easy_neg
# bigtom_task1_total = np.zeros((100, 3, 8))
for idx, instance in enumerate(task3_ans):
    if instance['groundtruth_classification'] == 'positive':
        for i in range(len(instance['llm_answer'])):
            # insert to hard array
            # llm_answer = instance['llm_answer'][i]
            llm_correct = instance['llm_answer'][i] == np.array(['A', 'B'])[np.array(instance['true_answer'])][i]
            converted_true_answer_total.append(1)
            converted_llm_answer_total.append(int(llm_correct))
    else: # negative
        for i in range(len(instance['llm_answer'])):
            llm_correct = instance['llm_answer'][i] == np.array(['A', 'B'])[np.array(instance['true_answer'])][i]
            converted_true_answer_total.append(0)
            converted_llm_answer_total.append(1-int(llm_correct))


print('positives (1) and all negative (0):\n', classification_report(converted_true_answer_total,converted_llm_answer_total, digits = 3))


positives (1) and all negative (0):
               precision    recall  f1-score   support

           0      0.396     0.703     0.507      1736
           1      0.847     0.606     0.706      4712

    accuracy                          0.632      6448
   macro avg      0.622     0.654     0.607      6448
weighted avg      0.726     0.632     0.653      6448



Task 4

In [199]:
bigtom_task4_ans = [{'ID': instance['ID'], 'groundtruth_classification': instance['groundtruth_classification']
                     , 'sense': instance['sense'],
                     'answer_order': [], # this is the f() that can map original order to postprocess order
                     'llm_answer': ['X','X','X','X','X','X','X','X','X','X'],
                     'true_answer': [],
                     'true_answer_og': option_dict[instance['new_belief_label']]} 
                    for instance in final_bigtom_with_noise]
opentom_task4_ans = [{'ID': instance['ID'], 'groundtruth_classification': instance['groundtruth_classification']
                     , 'sense': instance['sense'],
                     'answer_order': [], # this is the f() that can map original order to postprocess order
                     'llm_answer': ['X','X','X','X','X','X','X','X','X','X'],
                     'true_answer': [],
                     'true_answer_og': option_dict[instance['new_belief_label']]}  
                    for instance in final_opentom_with_noise]

In [200]:
import random
random.seed(0)
np.random.seed(0)
task4_ans = [bigtom_task4_ans, opentom_task4_ans]
for t in task4_ans:
    for instance in t:
        for _ in range(5):
            randomizer = np.random.choice(np.arange(4), 4, replace=False) # 0 --> a0, 1 --> a1
            instance['answer_order'].append(randomizer.tolist())
            # instance['true_answer'].append(['A','B','C','D'][randomizer[instance['true_answer_og']]])
            true_answer_idx = np.where(randomizer == instance['true_answer_og'])[0][0]
            instance['true_answer'].append(['A','B','C','D'][true_answer_idx])

In [201]:
for t in task4_ans:
    for instance in t:
        instance['answer_order'] = instance['answer_order'] + instance['answer_order']
        instance['true_answer'] = instance['true_answer'] + instance['true_answer']
        

In [204]:
save_jsonl(bigtom_task4_ans, f'bigtom_task4_ans_{model}.jsonl')

In [ ]:
# BigToM first - Task 1
model = "gpt-4o"
# positives
count = 0
input_tokens = 0
output_tokens = 0
begin = time.time()

# for i in range(20,50):
for i in range(len(bigtom_task4_ans)): # set to len(bigtom_task1_ans) later # 50 so far
    print(f'{i}-th story')
    for k in range(5):
        #story, event, protag_name, percept, initial_belief
        for j in range(2):
            story = final_bigtom_with_noise[i]['story']
            event = final_bigtom_with_noise[i]['event']
            protag_name = final_bigtom_with_noise[i]['protag_name']
            percept_signal = final_bigtom_with_noise[i]['percept_cue']
            # in each iteration, pick a different noise
            percept_noise = final_bigtom_with_noise[i]['noise'][k]
            
            if (j==0):
                percept = f"\n1. {percept_signal} \n2. {percept_noise}\n"
            else:
                percept = f"\n1. {percept_noise} \n2. {percept_signal}\n"
                
            initial_belief = final_bigtom_with_noise[i]['initial_belief']
            prompt = bigtom_system.format(story = story, event = event, protag_name = protag_name, percept = percept, initial_belief = initial_belief)
            
            default_question = final_bigtom_with_noise[i]['default_question']
            randomizer = np.array(bigtom_task4_ans[i]['answer_order'][k])
            # print(final_bigtom[i]['answer_options'])
            # print(randomizer)
            first = final_bigtom_with_noise[i]['answer_options'][randomizer[0]]
            second = final_bigtom_with_noise[i]['answer_options'][randomizer[1]]
            third = final_bigtom_with_noise[i]['answer_options'][randomizer[2]]
            fourth = final_bigtom_with_noise[i]['answer_options'][randomizer[3]]
            prompt += bigtom_q2_prompt.format(default_question=default_question, first=first, second=second, third=third, fourth=fourth)

            input_tokens += count_tokens(prompt)
            
            # print(prompt)
            # print("input tokens = ", count_tokens(prompt), ". price (USD) is ", count_tokens(prompt)*30/1000000)
            
            
            # Add back later
            output = LM_generation(prompt, temperature = 0, max_tokens = 3, model = model)
            llm_answer = output[0][0]
            bigtom_task4_ans[i]['llm_answer'][k + 5*j] = llm_answer

                    
            output_tokens += count_tokens(output[0])
            save_jsonl(bigtom_task4_ans, f'bigtom_task4_ans_{model}.jsonl')
            # print("output tokens = ", count_tokens(output[0]), ". price (USD) is ", count_tokens(output[0])*60/1000000)
                    
    if i % 5 == 0:
        end = time.time()
        print(f'Time taken for these 5 stories: {end-begin} seconds.')
        begin = time.time()


print("input tokens = ", input_tokens, ". price (USD) is ", input_tokens*30/1000000)
print("output tokens = ", output_tokens, ". price (USD) is ", output_tokens*60/1000000)





50-th story
Time taken for these 5 stories: 6.2777040004730225 seconds.
51-th story
52-th story
53-th story
54-th story
55-th story
Time taken for these 5 stories: 35.38343691825867 seconds.
56-th story
57-th story
58-th story
59-th story
60-th story
Time taken for these 5 stories: 65.64897012710571 seconds.
61-th story
62-th story
63-th story
64-th story
65-th story
Time taken for these 5 stories: 53.80545902252197 seconds.
66-th story
67-th story
68-th story
69-th story
70-th story
Time taken for these 5 stories: 56.78929781913757 seconds.
71-th story
72-th story
73-th story
74-th story
75-th story
Time taken for these 5 stories: 46.8396680355072 seconds.
76-th story
77-th story
78-th story
79-th story
80-th story
Time taken for these 5 stories: 52.904524087905884 seconds.
81-th story
82-th story
83-th story
84-th story
85-th story
Time taken for these 5 stories: 44.22372603416443 seconds.
86-th story
87-th story
88-th story
89-th story
90-th story
Time taken for these 5 stories: 57.

In [85]:
model = 'gpt-35-turbo'
with open(f'bigtom_task4_ans_{model}.jsonl') as f: # ambiguous story
    bigtom_task4_ans = [json.loads(line) for line in f]

In [86]:

bigtom_task4_llm_answer = [instance['llm_answer'][:3] + instance['llm_answer'][5:8] for instance in bigtom_task4_ans]
bigtom_task4_true_answer = [instance['true_answer'][:3] + instance['true_answer'][5:8] for instance in bigtom_task4_ans]
bigtom_task4_llm_answer = bigtom_task4_llm_answer # remove slicing later
bigtom_task4_true_answer = bigtom_task4_true_answer # remove slicing later
bigtom_task4_llm_answer = np.array(bigtom_task4_llm_answer)
bigtom_task4_true_answer = np.array(bigtom_task4_true_answer)
bigtom_task4_accuracy = (bigtom_task4_llm_answer == bigtom_task4_true_answer).mean(axis=1)


gt_id = {'positive':0, 'hard_negative': 1}
bigtom_task4_correct = np.zeros((100, 2, 8)) # groundtruth: pos, hard_neg, easy_neg
bigtom_task4_total = np.zeros((100, 2, 8))
for i, instance in enumerate(bigtom_task4_ans): # remove slicing later
    ider = instance['ID']
    story_idx = int(ider.split('-')[0])
    gt_idx = gt_id[instance['groundtruth_classification']]
    sense_idx = sense_dict[instance['sense']]
    bigtom_task4_total[story_idx][gt_idx][sense_idx] = 1
    bigtom_task4_correct[story_idx][gt_idx][sense_idx] = bigtom_task4_accuracy[i]
eps = 1e-10
print("TPR, TNR-hard: ", bigtom_task4_correct.sum(axis = 0).sum(axis = 1)/(bigtom_task4_total.sum(axis = 0).sum(axis = 1) + eps))

TPR, TNR-hard:  [0.89221273 0.00195312]


In [219]:
save_jsonl(opentom_task4_ans, f'opentom_task4_ans_{model}.jsonl')

In [220]:
# BigToM first - Task 1
model = "gpt-4o"
# positives
count = 0
input_tokens = 0
output_tokens = 0
begin = time.time()

# for i in range(2):
for i in range(len(opentom_task4_ans)): # set to len(bigtom_task1_ans) later # 50 so far
    print(f'{i}-th story')
    for k in range(5):
        #story, event, protag_name, percept, initial_belief
        for j in range(2):
            story = final_opentom_with_noise[i]['story']
            protag_name = final_opentom_with_noise[i]['protag_name']
            percept_signal = final_opentom_with_noise[i]['percept_cue']
            # in each iteration, pick a different noise
            percept_noise = final_opentom_with_noise[i]['noise'][k]
            
            if (j==0):
                percept = f"\n1. {percept_signal} \n2. {percept_noise}\n"
            else:
                percept = f"\n1. {percept_noise} \n2. {percept_signal}\n"
                
            initial_belief = final_opentom_with_noise[i]['initial_belief']
            prompt = opentom_system.format(story = story, protag_name = protag_name, percept = percept, initial_belief = initial_belief)
            
            default_question = final_opentom_with_noise[i]['default_question']
            randomizer = np.array(opentom_task4_ans[i]['answer_order'][k])
            # print(final_bigtom[i]['answer_options'])
            # print(randomizer)
            first = final_opentom_with_noise[i]['answer_options'][randomizer[0]]
            second = final_opentom_with_noise[i]['answer_options'][randomizer[1]]
            third = final_opentom_with_noise[i]['answer_options'][randomizer[2]]
            fourth = final_opentom_with_noise[i]['answer_options'][randomizer[3]]
            prompt += opentom_q2_prompt.format(default_question=default_question, first=first, second=second, third=third, fourth=fourth)

            input_tokens += count_tokens(prompt)
            
            # print(prompt)
            # print("input tokens = ", count_tokens(prompt), ". price (USD) is ", count_tokens(prompt)*30/1000000)
            
            
            # Add back later
            output = LM_generation(prompt, temperature = 0, max_tokens = 3, model = model)
            llm_answer = output[0][0]
            opentom_task4_ans[i]['llm_answer'][k + 5*j] = llm_answer

                    
            output_tokens += count_tokens(output[0])
            save_jsonl(opentom_task4_ans, f'opentom_task4_ans_{model}.jsonl')
            # print("output tokens = ", count_tokens(output[0]), ". price (USD) is ", count_tokens(output[0])*60/1000000)
                    
    if i % 5 == 0:
        end = time.time()
        print(f'Time taken for these 5 stories: {end-begin} seconds.')
        begin = time.time()


print("input tokens = ", input_tokens, ". price (USD) is ", input_tokens*30/1000000)
print("output tokens = ", output_tokens, ". price (USD) is ", output_tokens*60/1000000)





0-th story
Time taken for these 5 stories: 10.103092908859253 seconds.
1-th story
2-th story
3-th story
4-th story
5-th story
Time taken for these 5 stories: 33.46344995498657 seconds.
6-th story
7-th story
8-th story
9-th story
10-th story
Time taken for these 5 stories: 61.50483775138855 seconds.
11-th story
12-th story
13-th story
14-th story
15-th story
Time taken for these 5 stories: 50.05238699913025 seconds.
16-th story
17-th story
18-th story
19-th story
20-th story
Time taken for these 5 stories: 53.51764512062073 seconds.
21-th story
22-th story
23-th story
24-th story
25-th story
Time taken for these 5 stories: 51.84833812713623 seconds.
26-th story
27-th story
28-th story
29-th story
30-th story
Time taken for these 5 stories: 53.96471905708313 seconds.
31-th story
32-th story
33-th story
34-th story
35-th story
Time taken for these 5 stories: 50.06686782836914 seconds.
36-th story
37-th story
38-th story
39-th story
40-th story
Time taken for these 5 stories: 46.6030721664

In [94]:
model = 'gpt-35-turbo'
with open(f'opentom_task4_ans_{model}.jsonl') as f: # ambiguous story
    opentom_task4_ans = [json.loads(line) for line in f]

In [95]:

opentom_task4_llm_answer = [instance['llm_answer'][:3] + instance['llm_answer'][5:8] for instance in opentom_task4_ans]
opentom_task4_true_answer = [instance['true_answer'][:3] + instance['true_answer'][5:8] for instance in opentom_task4_ans]
opentom_task4_llm_answer = opentom_task4_llm_answer # remove slicing later
opentom_task4_true_answer = opentom_task4_true_answer # remove slicing later
opentom_task4_llm_answer = np.array(opentom_task4_llm_answer)
opentom_task4_true_answer = np.array(opentom_task4_true_answer)
opentom_task4_accuracy = (opentom_task4_llm_answer == opentom_task4_true_answer).mean(axis=1)


gt_id = {'positive':0, 'hard_negative': 1}
opentom_task4_correct = np.zeros((121, 2, 8)) # groundtruth: pos, hard_neg, easy_neg
opentom_task4_total = np.zeros((121, 2, 8))
for i, instance in enumerate(opentom_task4_ans): # remove slicing later
    ider = instance['ID']
    story_idx = int(ider.split('-')[0])
    gt_idx = gt_id[instance['groundtruth_classification']]
    sense_idx = sense_dict[instance['sense']]
    opentom_task4_total[story_idx][gt_idx][sense_idx] = 1
    opentom_task4_correct[story_idx][gt_idx][sense_idx] = opentom_task4_accuracy[i]
eps = 1e-10
print("TPR, TNR-hard: ", opentom_task4_correct.sum(axis = 0).sum(axis = 1)/(opentom_task4_total.sum(axis = 0).sum(axis = 1) + eps))

TPR, TNR-hard:  [0.93907233 0.05922166]


In [ ]:
# Supplementary - Template
import random
import json
import numpy as np
random.seed(0)

from sklearn.metrics import classification_report
model = 'llama-405b'
with open(f'opentom_task4_ans_{model}.jsonl') as f: # put story here
    task4_ans = [json.loads(line) for line in f]
    
letters = {'A':0, 'B':1, 'C':2, 'D':3}

task4_llm_answer = [instance['llm_answer'][:3] + instance['llm_answer'][5:8] for instance in task4_ans]
task4_true_answer = [instance['true_answer'][:3] + instance['true_answer'][5:8] for instance in task4_ans]
# bigtom_task1_llm_answer = bigtom_task1_llm_answer[0:50]
# bigtom_task1_true_answer = bigtom_task1_true_answer[0:50]
task4_llm_answer = np.array(task4_llm_answer)
task4_true_answer = np.array(task4_true_answer)

converted_llm_answer_total = []
converted_true_answer_total = []

# bigtom_task1_accuracy = (bigtom_task1_llm_answer == bigtom_task1_true_answer).mean(axis=1)
# gt_id = {'positive':0, 'hard_negative': 1, 'easy_negative':2}
# bigtom_task1_correct = np.zeros((100, 3, 8)) # groundtruth: pos, hard_neg, easy_neg
# bigtom_task1_total = np.zeros((100, 3, 8))
for idx, instance in enumerate(task4_ans):
    for i in [0,1,2,5,6,7]:
        true_answer = instance['true_answer_og']
        llm_answer_index = instance['llm_answer'][i]
        if llm_answer_index not in ['A','B','C','D']:
            llm_answer_index = random.choice(['A','B','C','D'])
        llm_answer_index = letters[llm_answer_index]
        llm_answer = instance['answer_order'][i][llm_answer_index]
        converted_true_answer_total.append(true_answer)
        converted_llm_answer_total.append(llm_answer)

print('positives (1) and all negative (0):\n', classification_report(converted_true_answer_total,converted_llm_answer_total, digits = 3,labels=[0,1,2,3]))
